In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
df_train.head()

In [ ]:
X_train, y_train = df_train.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1), df_train['Survived']
# X_test, y_test = df_test.drop('Survived', axis=1), df_test['Survived']

In [ ]:
X_train.info()

In [ ]:
X_train = X_train.drop('Cabin', axis=1)

In [ ]:
X_train.describe()

In [ ]:
# import seaborn as sns

# sns.boxplot(X_train)

In [ ]:
# sns.pairplot(X_train)

In [ ]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean')
# X_train['Age'] = imputer.fit_transform(X_train['Age'])

In [ ]:
X_train = X_train.dropna()

In [ ]:
X_train.info()

In [ ]:
X_train.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded = encoder.fit_transform(X_train[['Sex', 'Embarked']])

In [ ]:
encoded = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Sex', 'Embarked']), index=X_train.index)

In [ ]:
X_train = pd.concat([X_train.drop(['Sex', 'Embarked'], axis=1), encoded], axis=1)

In [ ]:
X_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_cls = DecisionTreeClassifier()

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_cls, X_train, y_train, cv=5, scoring='neg_mean_squared_error')